In [ ]:
import pandas as pd
import numpy as np

import open3d as o3d
import matplotlib.pyplot as plt
from numpy import dtype

In [ ]:
df = pd.read_csv('pixel_digis.csv')
df.shape

In [ ]:
events = df.event.unique()
events.shape

In [ ]:
df[df.event == events[0]]

In [ ]:
cmap = plt.get_cmap('tab20')  # 20 distinct colors repeated if needed

point_cloud = o3d.geometry.PointCloud()
points = []
colors = []

In [ ]:
events = df.event.unique()[:200]
events.shape

In [ ]:
for idx, event in enumerate(events):
    sub_df = df[df.event == event]
    pts = sub_df[['global_x', 'global_y', 'global_z']].values
    color = cmap(idx % 20)[:3]  # RGB tuple, ignore alpha
    color_array = np.tile(color, (pts.shape[0], 1))

    points.append(pts)
    colors.append(color_array)

In [ ]:
# Stack all event points and colors
points = np.vstack(points)
colors = np.vstack(colors)

In [ ]:
# Load into open3d
point_cloud.points = o3d.utility.Vector3dVector(points)
point_cloud.colors = o3d.utility.Vector3dVector(colors)

In [ ]:
tolerance = 1e-4

x_diffs, y_diffs, z_diffs = np.diff(np.sort(df.global_x.unique())), np.diff(np.sort(df.global_y.unique())), np.diff(np.sort(df.global_z.unique()))
min_x, min_y, min_z = np.min(x_diffs[x_diffs > tolerance]), np.min(y_diffs[y_diffs > tolerance]), np.min(z_diffs[z_diffs > tolerance])
min_x, min_y, min_z

In [ ]:
o3d.visualization.draw_geometries([point_cloud])

In [ ]:
voxel_size = 0.0001
voxels = point_cloud.voxel_down_sample(voxel_size=voxel_size)

In [ ]:
o3d.visualization.draw_geometries([voxels])

In [ ]:
np.asarray(point_cloud.points).shape, np.asarray(voxels.points).shape

In [ ]:
min_bound = point_cloud.get_min_bound()
max_bound = point_cloud.get_max_bound()

dims = np.ceil((max_bound - min_bound) / voxel_size).astype(int)
dims

In [ ]:
df.groupby('event')['detId'].count().hist(bins=50)

In [ ]:
df.groupby('event')['detId'].count().min(), df.groupby('event')['detId'].count().max()